In [28]:
from dotenv import load_dotenv
import os

from typing import TypedDict, Annotated
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

load_dotenv()


True

In [29]:
llm = init_chat_model("openai:gpt-4.1-nano")


In [30]:
# llm.invoke("Who's the first to walk on moon?")


In [31]:
class ChatState(TypedDict):
    messages: Annotated[list, add_messages]


def chatbot(state: ChatState):
    return {"messages": [llm.invoke(state["messages"])]}


builder = StateGraph(ChatState)

builder.add_node("chatbot", chatbot)

builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()


In [32]:
message = {
    "role": "user",
    "content": "Who's the first to walk on moon? Just give the name only.",
}

response = graph.invoke({"messages": [message]})
response["messages"]


[HumanMessage(content="Who's the first to walk on moon? Just give the name only.", additional_kwargs={}, response_metadata={}, id='535f51ba-b0a1-4589-95b1-a4627bb0e580'),
 AIMessage(content='Armstrong', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 21, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-C8ATPjeFJORS06Y5XrPinaoqOQpB9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--569969b1-b734-4ddd-8505-0cac2a005ff3-0', usage_metadata={'input_tokens': 21, 'output_tokens': 2, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [35]:
state = None
while True:
    in_message = input("Enter your message: ")
    if in_message.lower() in ["exit", "quit"]:
        break
    if state is None:
        state: ChatState = {
            "messages": [
                {
                    "role": "user",
                    "content": in_message,
                }
            ]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})
    state = graph.invoke(state)
    print("Bot: ", state["messages"][-1].content)


Bot:  The first man to land on the Moon was Neil Armstrong. He made his historic "first step" on July 20, 1969, during the Apollo 11 mission.
Bot:  The second person to land on the Moon was Buzz Aldrin. He walked on the lunar surface shortly after Neil Armstrong during the Apollo 11 mission on July 20, 1969.
Bot:  During the Apollo 11 mission, Michael Collins was orbiting the Moon in the Command Module. While Neil Armstrong and Buzz Aldrin were on the lunar surface, Collins remained in orbit around the Moon, overseeing the command module and ensuring the spacecraft's stability and readiness for the astronauts' return.
Bot:  While there are no detailed personal accounts of Michael Collins expressing specific feelings about not landing on the Moon, he has publicly spoken about his thoughts and experiences. Collins has expressed pride in his role and the significance of the mission as a whole. In interviews, he has acknowledged that while he didn’t get to walk on the lunar surface, he fel